# Mindguard IQ Analysis Notebook

Simulated, local-only analysis pipeline that:
- Loads interleaved float32 IQ (.iq) or .npy complex IQ
- Computes and plots a full-capture Power Spectral Density (PSD)
- Generates a waterfall (spectrogram) image
- Maps probe positions to a near-field heatmap using snapshot band-power
- Saves results into `runs/<run_id>/analysis`

Safety note: This notebook uses simulated IQ files for contained analysis. Do not transmit these files over the air unless you are inside a certified Faraday cage and have explicit authorization.

In [ ]:
# Cell 1: Imports and helper functions
import os
import sys
import json
from datetime import datetime
import numpy as np
import yaml
import matplotlib.pyplot as plt
from scipy.signal import welch, spectrogram

plt.style.use('seaborn-v0_8-darkgrid')

def load_iq(path):
    if path.endswith('.npy'):
        arr = np.load(path)
        return arr.astype(np.complex64)
    else:
        data = np.fromfile(path, dtype=np.float32)
        if data.size % 2 != 0:
            raise ValueError('IQ file length not even; expecting interleaved float32 I,Q')
        i = data[0::2]
        q = data[1::2]
        return (i + 1j*q).astype(np.complex64)

def compute_psd(iq, fs, nperseg=4096):
    f, Pxx = welch(iq, fs=fs, nperseg=nperseg, return_onesided=True)
    Pdb = 10 * np.log10(Pxx + 1e-20)
    return f, Pdb

def make_waterfall(iq, fs, outpath=None, nperseg=2048, noverlap=1024, cmap='viridis'):
    f, t, Sxx = spectrogram(iq, fs=fs, nperseg=nperseg, noverlap=noverlap)
    Sdb = 10 * np.log10(Sxx + 1e-20)
    fig, ax = plt.subplots(figsize=(10,4))
    pcm = ax.pcolormesh(t, f, Sdb, shading='gouraud', cmap=cmap)
    ax.set_ylabel('Freq [Hz]')
    ax.set_xlabel('Time [s]')
    ax.set_title('Waterfall (spectrogram)')
    fig.colorbar(pcm, ax=ax, label='PSD (dB)')
    fig.tight_layout()
    if outpath:
        fig.savefig(outpath, dpi=150)
    return fig, ax

def band_power(iq_segment, fs, band):
    # Compute PSD on the segment and average power in band
    f, Pdb = compute_psd(iq_segment, fs)
    mask = (f >= band[0]) & (f <= band[1])
    if not np.any(mask):
        return float('-inf')
    return float(np.mean(Pdb[mask]))

def ensure_dir(p):
    os.makedirs(p, exist_ok=True)


In [ ]:
# Cell 2: Parameters and paths (edit if needed)
MANIFEST_PATH = 'experiments/manifests/example_manifest.yaml'
SIM_IQ_REL = 'simulated_captures/mindguard_sim_001.iq'
RUNS_DIR = 'runs'

# Load manifest
with open(MANIFEST_PATH, 'r') as f:
    manifest = yaml.safe_load(f)

experiment_id = manifest.get('experiment_id', 'run')
run_id = experiment_id + '_' + datetime.utcnow().strftime('%Y%m%dT%H%M%SZ')
out_dir = os.path.join(RUNS_DIR, run_id)
analysis_dir = os.path.join(out_dir, 'analysis')
ensure_dir(analysis_dir)

print('Run ID:', run_id)
print('Analysis directory:', analysis_dir)


In [ ]:
# Cell 3: Load IQ data (simulated capture)
iq_path = os.path.join(out_dir, SIM_IQ_REL)
# The orchestrator usually places the simulated IQ under the run folder; fall back to top-level path
if not os.path.exists(iq_path):
    if os.path.exists(SIM_IQ_REL):
        iq_path = SIM_IQ_REL
    else:
        raise FileNotFoundError(f"Cannot find IQ file at {iq_path} or {SIM_IQ_REL}")

print('Loading IQ from:', iq_path)
iq = load_iq(iq_path)
fs = float(manifest['capture_parameters']['samplerate_hz'])
print('IQ samples:', len(iq), 'Samplerate (Hz):', fs)


In [ ]:
# Cell 4: Compute and plot full-capture PSD
f, Pdb = compute_psd(iq, fs)
plt.figure(figsize=(10,3))
plt.plot(f, Pdb)
plt.title('PSD (full capture)')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Power (dB)')
plt.tight_layout()
psd_file = os.path.join(analysis_dir, 'psd_full.png')
plt.savefig(psd_file, dpi=150)
plt.show()
print(f'[+] Saved PSD to {psd_file}')


In [ ]:
# Cell 5: Waterfall (spectrogram)
waterfall_file = os.path.join(analysis_dir, 'waterfall.png')
fig, ax = make_waterfall(iq, fs, outpath=waterfall_file)
plt.show()
print(f'[+] Saved waterfall to {waterfall_file}')


In [ ]:
# Cell 6: Near-field heatmap via snapshot band-power mapping
probe_positions = manifest['probe_grid']['positions']
snapshots = manifest['capture_parameters'].get('snapshots', 5)
n = len(iq)
seg = max(1, n // snapshots)
center = manifest['capture_parameters'].get('center_freq_hz', 0)
span = manifest['capture_parameters'].get('freq_span_hz', 1e6)
band = (center - span/2, center + span/2)

powers = []
for i in range(snapshots):
    start = i * seg
    stop = min(n, (i+1) * seg)
    seg_iq = iq[start:stop]
    p = band_power(seg_iq, fs, band)
    powers.append(p)

# Map snapshot powers to probe positions (wrap if counts differ)
vals = np.array([powers[i % len(powers)] for i in range(len(probe_positions))])
xs = np.array([p[0] for p in probe_positions])
ys = np.array([p[1] for p in probe_positions])

plt.figure(figsize=(6,4))
sc = plt.scatter(xs, ys, c=vals, cmap='inferno', s=300, edgecolor='k')
plt.gca().invert_yaxis()
plt.colorbar(sc, label='Band power (dB)')
plt.title('Near-field heatmap (simulated mapping)')
plt.xlabel('X mm')
plt.ylabel('Y mm')
heatmap_file = os.path.join(analysis_dir, 'nearfield_heatmap.png')
plt.tight_layout()
plt.savefig(heatmap_file, dpi=150)
plt.show()
print(f'[+] Saved near-field heatmap to {heatmap_file}')


In [ ]:
# Cell 7: Save summary metadata and notes
summary = {
    'psd_image': os.path.basename(psd_file),
    'waterfall_image': os.path.basename(waterfall_file),
    'heatmap_image': os.path.basename(heatmap_file),
    'iq_file': os.path.basename(iq_path),
    'samples': int(len(iq)),
    'samplerate_hz': fs,
    'generated_at': datetime.utcnow().isoformat() + 'Z',
    'safety_note': 'Simulated IQ used. Do not transmit offline files over the air unless inside certified shielded enclosure and authorized.'
}
with open(os.path.join(analysis_dir, 'analysis_summary.json'), 'w') as f:
    json.dump(summary, f, indent=2)
print('[+] Wrote analysis_summary.json')
print('\nRun complete. Review the PNGs in the analysis directory for PSD, waterfall, and heatmap visualizations.')
